# 1. Extracción del texto de un discurso 

In [ ]:
# Instalación de la librería pandas
import sys
!{sys.executable} -m pip install pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
url = 'https://elpais.com/internacional/2018/01/31/actualidad/1517387619_036241.html'
html = requests.get(url)
soup2 = BeautifulSoup(html)

tags = soup2('p') # Extraigo las etiquetas de tipo <p>
discurso = '' # Aquí guardo el discurso, inicialmente vacío

for tag in tags:
    if (len(tag.attrs)) == 0: # Con esta condición filtro solo el texto que me interesa
        a= tag.contents[0] # Extraigo el contenido de la etiqueta
        discurso = discurso +  a # Voy concatenando cada contenido a la variable discurso

#print(discurso)

contadores = dict() # Creo un diccionario para almacenar las palabras y contar las veces que aparece
discurso = discurso.replace(',','').replace('.','').replace(':','').replace('?','').replace('(','').replace(')','') # Elimino signos de (puntos, comas, etc.)
palabras = discurso.lower().split() # Paso todas las palabras a minúsculas (lower) y las corto palabra a palabra (split)
for palabra in palabras:
    if len(palabra)>3: # Añado al diccionario solo las palabras con más de 3 letras
        contadores[palabra] = contadores.get(palabra,0) + 1 
        
        
import pandas as pd
# Guardo las palabras en un dataframe ordenados de mayor a menor por el contador
pd.DataFrame(list(contadores.items()), columns=['palabra', 'contador']).sort_values('contador', ascending=False )


# 2. Extracción de valores de un fondo de inversión 

Para extraer datos de una web tenemos que analizar el código fuente. Si inspeccionamos el valor del fondo vemos que tiene esta etiqueta: 
```html
<td class="line text">EUR 15.15</td>
```

Por lo tanto, voy a utilizar BeautifulSoup para extraer las etiquetas `"td"` con `class = "line text"`

In [2]:
import urllib
from bs4 import BeautifulSoup
#iShares Dev Rl Ett Idx (IE) Instl Acc €
participaciones = 50 # Estas son mis participaciones en el fondo de inversión
url= 'http://www.morningstarfunds.ie/ie/funds/snapshot/snapshot.aspx?id=F00000PJME'
html=urllib.request.urlopen(url)
soup=BeautifulSoup(html)
tags = soup.find_all("td",class_="line text") # Extraigo las etiquetas 
valor = float(tags[0].contents[0].replace('EUR\xa0','')) # Me quedo únicamente con el valor numérico
total = participaciones * valor
print(total)

849.0


# 3. Web scraping de laliga.es

Extraemos los hashtags de los próximos encuentros  
<img src='partidos2.jpg'>

In [ ]:
import urllib
import re
from bs4 import BeautifulSoup

url = 'https://www.laliga.com/laliga-santander/clubes'
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html)

# Si inspeccionamos la página podemos ver cómo esa clase es la asignada a cada equipo
#tags = soup.find_all("h2",class_="styled__TextHeaderStyled-sc-1edycnf-0 bYHKtg") #El problema de esta solución, es que busca un atributo específico
regex = re.compile('styled__TextHeaderStyled-sc-1edycnf-0')  
tags = soup.find_all("h2", {"class": regex}) #Esta solución, puede buscar una coincidencia parcial

hashtags = list()
teams = dict()

for tag in tags:
    hashtags.append(tag.contents[0])
   # print(tag.find_parent('a'))
    tagParentTeam = tag.find_parent('a')
    print(tagParentTeam['href'])
    teams[tag.contents[0]] = tagParentTeam['href']


#print(hashtags)
#print(teams)

In [ ]:
# Para navegar a cada equipo, se necesita obtener la url-del-equipoo
urlbase = 'https://www.laliga.com'
for teamName,urlTeamPartial in teams.items():
    urlTeam = urlbase+urlTeamPartial;
    print(urlTeam)

    

In [ ]:
# Procesar cada equipo
 # definir una función para procesar cada equipo